In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../../')
import src.results as results
import pandas as pd
from math import sqrt

d:\ProgramData\Anaconda3\envs\rwn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def agg_res(res, name=None):
    new_res = {'classification':{}, 'link_pred':{}}

    for agg, runs in res.items():
        new_res[agg] = {}
        classification = []
        link_pred = []
        for r in runs:
            classification.append(r[0][2])
            link_pred.append(float(r[0][1].cpu().numpy()))
        new_res['classification'][agg] = classification
        new_res['link_pred'][agg] = link_pred

    if name is not None:
        print('=' * 50, f' {name.upper()} ', '='*50)
    print('Classification Acc')
    df = pd.DataFrame(new_res['classification'])
    display(df.mean())
    print()
    display(df.std())

    print('-'*100)

    print('Link Pred AUC')
    df = pd.DataFrame(new_res['link_pred'])
    display(df.mean())
    print()
    display(df.std())
    print()
    print()

    return new_res


In [4]:
funcs = [
    lambda x, y: 0.5,
    lambda x, y: 0.9,
    lambda x, y: 0.1,
    lambda x, y: 0.25,
    lambda x, y: 0.75,
    lambda x, y: x / y,

    # These next three have no rationale but fuck it who cares
    lambda x, y: (x / y) ** 2,
    lambda x, y: sqrt(x / y),
    lambda x, y: y / x
    ]

In [8]:
res = []
for i, l in enumerate(funcs):
    x = results.run_joint_train('Cora', 3, epochs=1000, lamb=l)
    res.append(agg_res(x, name=f'Lambda #{i}'))

100%|██████████| 3/3 [00:58<00:00, 19.41s/it]

==================================================  LAMBDA #0  ==================================================
Classification Acc


eve     0.511388
mean    0.857444
gcn     0.856145
pool    0.754040
dtype: float64

eve     0.015432
mean    0.004534
gcn     0.018859
pool    0.002258
dtype: float64

----------------------------------------------------------------------------------------------------
Link Pred AUC


eve     0.091333
mean    0.772333
gcn     0.755333
pool    0.622000
dtype: float64

eve     0.001528
mean    0.014048
gcn     0.007095
pool    0.047634
dtype: float64

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:

res_pubmed = []
for i, l in enumerate(funcs):
    x = results.run_joint_train('PubMed', 3, epochs=1000, lamb=l)
    res_pubmed.append(agg_res(x, name=f'Lambda #{i}'))

In [ ]:
res_citeseer = []
for i, l in enumerate(funcs):
    x = results.run_joint_train('CiteSeer', 3, epochs=1000, lamb=l)
    res_citeseer.append(agg_res(x, name=f'Lambda #{i}'))